### Objectif: 
#### Extraire les informations pertinentes à partir du document de spécifications fonctionnelles, et les insérer dans la base de données mysql, en utilisant du NLP

In [1]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import csv
import nltk
import re
import sys
import time
import os
from enum import Enum
from difflib import SequenceMatcher
import os.path
from os.path import basename
import mysql.connector

config = {
  'user': 'root',
  'password': '',
  'host': '127.0.0.1',
  'database': 'gottra',
  'raise_on_warnings': True,
}

import treetaggerwrapper as tt

In [2]:
#fonctions pour l'interaction avec la bdd

def add_case(name, title, ID_fichier):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("INSERT INTO usecase (nom, titre, id_fichier) VALUES (%s, %s, %s)")        
    cursor.execute(query, (name, title, ID_fichier))
    id = cursor.lastrowid    
    cnx.commit()
    cursor.close()
    cnx.close()
    return id

def add_actor(acteur):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("INSERT INTO acteur (nom) VALUES (%s)")
    cursor.execute(query, (acteur,))
    ID_acteur = cursor.lastrowid
    cnx.commit()
    cursor.close()
    cnx.close()
    return ID_acteur

def add_action(action, output, ID_cas, full_text):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("INSERT INTO action (ID_cas, action, output, full_text) VALUES (%s, %s, %s, %s)")
    cursor.execute(query, (ID_cas, action, output, full_text))
    cnx.commit()
    cursor.close()
    cnx.close()

def add_rule(text, ID_cas):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("INSERT INTO rgestion (ID_cas, texte) VALUES (%s, %s)")
    cursor.execute(query, (ID_cas, text))
    cnx.commit()
    cursor.close()
    cnx.close()

def actor_exists(acteur):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("SELECT ID_acteur FROM acteur WHERE nom = %s")
    cursor.execute(query, (acteur,))
    ID = None
    for ID in cursor:
        break;
    cursor.close()
    cnx.close()    
    return ID

def case_exists(ID_cas, titre):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("SELECT ID_cas FROM usecase WHERE ID_cas = %s AND titre = %s")
    cursor.execute(query, (ID_cas, titre))
    ID = None
    for ID in cursor:
        break;
    cursor.close()
    cnx.close()
    return ID

def update_case(ID_cas, ID_acteur):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    query = ("UPDATE usecase SET ID_acteur = (%s) WHERE ID_cas = (%s)")
    cursor.execute(query, (ID_acteur, ID_cas))
    cnx.commit()
    cursor.close()
    cnx.close()

In [3]:
#fonction de similarité syntaxique entre deux mots 
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

#retourne le premier verbe rencontré dans une phrase, sinon une chaîne de caractères vide
def get_verb(sentence):
    sentence = sentence.replace("'", " ")
    tt_fr = tt.TreeTagger(TAGLANG='fr',TAGDIR='C:/TreeTagger')
    tags = tt_fr.TagText(sentence)
    print(sentence)
    
    for word in tags:
        if(len(word.split("\t")) > 1):
            POS = word.split("\t")[1]
            word = word.split("\t")[2]
            if("VER" in POS):
                with open('../Ressources/verbes_cs.txt') as actions:
                    print("verbe: " + word)
                    word_ = word.upper().replace(' ', '')
                    sims = [(similar(word_, action), action) for action in actions]
                    res = max(sims)
                    if(res[0] > 0.8):
                        print(word_ + ' est similaire à ' + res[1] + "Score: " + str(res[0]) + "\n")
                        return word
    return ''

#à partir d'une phrase typique dans les paragraphes décrivant les actions possible d'un cas d'utilisation, 
#retourner la partie de la phrase décrivant l'action, et celle décrivant l'output
def get_action_output(sentence):
    split = sentence.split(",")
    if(len(split) > 1):
        action = split[:len(split)-1]
        action = ', '.join(split[:len(split)-1])
        output = str(split[len(split)-1])
    elif(get_verb(sentence)):
        action = sentence[:sentence.find(get_verb(sentence))]
        output = sentence[sentence.find(get_verb(sentence)):]
    else:
        action = sentence
        output = ""
    return (action, output)

#fonction principale faisant le traitement de l'extraction à partir d'un fichier donné et en utilisant les autres fonctions définies ci-dessus
def alimenter_bdd(file, ID_fichier):
    start_time = time.time()
    regex_cas = re.compile(r'[a-zA-Z]+\d+-\d+')
    with open(file, encoding = "UTF-8") as f:
        with open('../Ressources/'+os.path.splitext(basename(file))[0]+'.csv', 'w+', newline = '') as writer:
            mywriter = csv.writer(writer, delimiter = ";")
            mywriter.writerow(["texte", "tag"])
            action = False
            rgestion = False
            text = False
            text_ = ''
            rgestion_text = ''
            action_text = ''
            for sentence in f:
                sentence = sentence.strip()
                if(action):
                    rgestion_text = ''
                    if( (sentence == "\n") | (sentence == "") ):
                        mywriter.writerow([action_text, "action"])
                        action = False
                    else:
                        action_text = action_text + '\n' + sentence
                        action, output = get_action_output(sentence)
                        add_action(action, output, ID_cas, sentence)
                        
                elif(rgestion):
                    action_text = ''
                    text_ = ''
                    if( (sentence == "\n") | (sentence == "") ):
                        mywriter.writerow([rgestion_text, "gestion"])
                        rgestion = False
                        add_rule(rgestion_text, ID_cas)
                    else:
                        rgestion_text = rgestion_text  + '\n' + sentence
                
                elif(text):
                    if( (sentence == "\n") | (sentence == "") ):
                        mywriter.writerow([text_, "texte"])
                        text = False
                    else:
                        text_ = text_ + '\n' + sentence
                    
                else:
                    action_text = ''
                    rgestion_text = ''
                    text_ = ''
                    reg = regex_cas.search(sentence)
                    if(reg):
                        position = reg.span()
                        name_cas = sentence[position[0]:position[1]].strip()
                        title_cas = sentence[position[1]:None].strip()
                        ID_cas = add_case(name_cas, title_cas, ID_fichier)
                        mywriter.writerow([sentence, "usecase"])
                        
                    elif(sentence.lower().startswith('acteur')):
                        mywriter.writerow([sentence, "acteur"])
                        sentence = sentence.strip()[9:None]
                        for actor_ in sentence.split('/'):
                            for actor in actor_.split('&'):
                                actor = actor.strip()
                                ID_acteur = actor_exists(actor)
                                if(ID_acteur == None):
                                    ID_acteur = add_actor(actor)
                                else:
                                    ID_acteur = ID_acteur[0]
                                update_case(ID_cas, ID_acteur)

                    elif('actions' in sentence.lower()):
                        action = True

                    elif(('règles de gestion' in sentence.lower()) | ('règles gestion' in sentence.lower()) ):
                        rgestion = True
                        
                    else:
                        if((sentence != "\n") & (sentence != "")):
                            text_ = sentence
                            text = True

In [4]:
#specFile = "C:\\Users\\fkafou\\Documents\\GOTTRA++\\Docs\\specs fonctionnelles.txt"
specFile = '../Ressources/specs fonctionnelles.txt'
ID_fichier = 30

alimenter_bdd(specFile, ID_fichier)

Si l’utilisateur clique
Un clic sur un bouton de réseaux sociaux permet à l’utilisateur d’accéder au réseau social concerné (dans une nouvelle fenêtre) :
verbe: permettre
PERMETTRE est similaire à EMETTRE
Score: 0.8235294117647058

Un clic sur un bouton de réseaux sociaux permet à l’utilisateur d’accéder au réseau social concerné (dans une nouvelle fenêtre) :
verbe: permettre
PERMETTRE est similaire à EMETTRE
Score: 0.8235294117647058

Un clic sur un bouton de réseaux sociaux permet à l’utilisateur d’accéder au réseau social concerné (dans une nouvelle fenêtre) :
verbe: permettre
PERMETTRE est similaire à EMETTRE
Score: 0.8235294117647058

Un clic sur le bouton facebook redirige vers la page suivante :
verbe: redirige
REDIRIGE est similaire à REDIRIGE
Score: 0.9411764705882353

Un clic sur le bouton facebook redirige vers la page suivante :
verbe: redirige
REDIRIGE est similaire à REDIRIGE
Score: 0.9411764705882353

Un clic sur le bouton facebook redirige vers la page suivante :
verbe:

REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

L’internaute est redirigé vers la page de l’évènement au clic sur le bloc de contenu  ou sur les éléments suivants :
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

L’image
Le titre
La description
Le bouton En savoir plus
verbe: En
verbe: savoir
L’internaute est redirigé vers la page de l’évènement au clic sur le bloc de contenu  ou sur les éléments suivants :
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

L’internaute est redirigé vers la page de l’évènement au clic sur le bloc de contenu  ou sur les éléments suivants :
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

L’internaute est redirigé vers la page de l’évènement au clic sur le bloc de contenu  ou sur les éléments suivants :
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

L’image
Le titre
La description
Le

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Libellé de partage de la rubrique audio + Titre + lien raccourci+ Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé de partage de la rubrique audio + Titre + lien raccourci+ Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé de partage de la rubrique audio + Titre + lien raccourci+ Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Au clic sur le bouton Partager de Facebook. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Facebook. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Au

PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Libellé de partage de la rubrique Accessibilité  + Titre + lien raccourci + Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé de partage de la rubrique Accessibilité  + Titre + lien raccourci + Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé de partage de la rubrique Accessibilité  + Titre + lien raccourci + Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Au clic sur le bouton Partager de Facebook. Les éléments suivants sont partagés
v

•	Un clic sur le lien affiche le contenu de la facette Personnes (voir cas d’utilisation)
verbe: Un
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

•	Un clic sur le lien affiche le contenu de la facette Cultures (voir cas d’utilisation)
verbe: Un
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

•	Un clic sur le lien affiche le contenu de la facette Cultures (voir cas d’utilisation)
verbe: Un
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

•	Un clic sur le lien affiche le contenu de la facette Cultures (voir cas d’utilisation)
verbe: Un
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

•	Un clic sur le lien affiche le contenu de la facette Pays (voir cas d’utilisation)
verbe: Un
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

•	Un clic sur le lien affiche le contenu de la facette Pays (voir cas d’utilisation)
verbe: Un
verbe: afficher
AFFICHER est

...

AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

Un clic sur la flèche précédente affiche l’image précédente en lieu est place de l’image en cours de consultation
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

Un clic sur la flèche suivante affiche l’image suivante en lieu est place de l’image en cours de consultation
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

Un clic sur la flèche suivante affiche l’image suivante en lieu est place de l’image en cours de consultation
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

Un clic sur la flèche suivante affiche l’image suivante en lieu est place de l’image en cours de consultation
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

•	Un clic sur la flèche précédente renvoie l’internaute vers la notice précédente
verbe: Un
verbe: renvoyer
RENVOYER est similaire à ENVOYER
Score: 0.875

•	Un clic sur la flèche précéden

Au swipe avec le doigt de droite à gauche
Le nombre d’éléments affichés est en fonction de la largeur de la fenêtre du navigateur
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

Le nombre d’éléments affichés est en fonction de la largeur de la fenêtre du navigateur
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

Le nombre d’éléments affichés est en fonction de la largeur de la fenêtre du navigateur
verbe: afficher
AFFICHER est similaire à AFFICHER
Score: 0.9411764705882353

l’internaute est redirigé vers la page de l’évènement au clic sur
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

l’internaute est redirigé vers la page de l’évènement au clic sur
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

l’internaute est redirigé vers la page de l’évènement au clic sur
verbe: être
verbe: redirigé
REDIRIGÉ est similaire à REDIRIGE
Score: 0.8235294117647058

EFFECTUER est similaire à EFFECTEUR
Score: 0.8421052631578947

L’utilisateur a effectué une modification dans la date de début ou la date de fin du menu
verbe: avoir
verbe: effectuer
EFFECTUER est similaire à EFFECTEUR
Score: 0.8421052631578947

L’utilisateur a effectué une modification dans la date de début ou la date de fin du menu
verbe: avoir
verbe: effectuer
EFFECTUER est similaire à EFFECTEUR
Score: 0.8421052631578947

L’utilisateur a modifié les filtres de recherche
verbe: avoir
verbe: modifier
MODIFIER est similaire à MODIFIER
Score: 0.9411764705882353

L’utilisateur a modifié les filtres de recherche
verbe: avoir
verbe: modifier
MODIFIER est similaire à MODIFIER
Score: 0.9411764705882353

L’utilisateur a modifié les filtres de recherche
verbe: avoir
verbe: modifier
MODIFIER est similaire à MODIFIER
Score: 0.9411764705882353

Action sur l’événement :
Action sur l’icône horaire de l’événement :
Au clic sur l’icône horaire :
•	Sinon le système affiche
Au clic sur le bouton Tarifs

PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Libellé de partage de la rubrique Photo  + Titre + lien raccourci + Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé de partage de la rubrique Photo  + Titre + lien raccourci + Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé de partage de la rubrique Photo  + Titre + lien raccourci + Champ texte libre twitter
verbe: libeller
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

On peut retourner à la page média d’origine via la page précédente
verbe: pouvoir
verbe: retourner
RETOUR

Lien qui pointe vers la page de démarrage du parcours
verbe: pointer
POINTER est similaire à POINTEUR
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Au clic sur le bouton Partager de Twitter. Les éléments suivants sont partagés
verbe: être
verbe: partager
PARTAGER est similaire à PARTAGE
Score: 0.875

Libellé Champ de partage twitter    + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
verbe: twitter
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé Champ de partage twitter    + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
verbe: twitter
verbe: titrer
TITRER est similaire à TITRE
Score: 0.8333333333333334

Libellé Cham